# [ **RUN ALWAYS** ] Import, initialize

In [ ]:
import math
import geemap
import ee
import configparser as cp
import io
import ast
import subprocess

from classification import classifyAndAssess
from features import ecological

ee.Initialize(project = 'ee-open-natural-ecosystems')

Find paths to the home folder and results for this run.

In [2]:
with open("config.ini", 'r') as f:
    fileContents = f.read()
config = cp.RawConfigParser(allow_no_value = True, interpolation = cp.ExtendedInterpolation())
config.read_file(io.StringIO(fileContents))

configCore = config["CORE"]
homeFolder = configCore.get('assetFolderWithFeatures')
configClassify = config["CLASSIFICATION-TRAIN&PREDICT"]
resColl = configClassify.get('existingPredictionsCollection')

## Feature names from saved feature rasters

Read the names of all the features associated with each labeled training point. Verify that the set of features is as intended.

The function `assembleAllExistingFeatureRasters()` in the `generateFeatures` module gathers all relevant rasters from the config-designated folder of EE assets.
It "manually" reads-in all feature rasters using their generation functions, but with `returnExisting` flag set to `True`.


In [3]:
namesOfAllFeatures = ecological.assembleAllExistingFeatureRasters('2019').bandNames().getInfo()
print(namesOfAllFeatures)


['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63', 'ae_regcode', 'longitude', 'latitude']


## Classification

Select the features we will use, choose the classifier and set its parameters.

In [4]:
selFeatureNames = [
    'A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15',
    'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31',
    'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47',
    'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63',
    'ae_regcode'] # 'longitude', 'latitude'

cOpts_gbt150 = dict(classifier = "GradientBoostedTrees", numTrees = 150, maxNodes = 500, trainFraction = 0.7)
fOpts_globalModelGbt_agroecolZones = dict(names = selFeatureNames, zonationBasis = "agroecol", zoneEncodingMode = "num")


### Classify to predict final labels

Create a folder for the results.

In [5]:
resFolderAgroecolZonesNum_finalPred = "yearwiseClassify_AgroecolZonesNum_finalPred_fullAoi"
createResFolderCmd = f"earthengine create folder {homeFolder}{resFolderAgroecolZonesNum_finalPred}"
process = subprocess.Popen(createResFolderCmd.split(), stdout=subprocess.PIPE)
folderCreated, error = process.communicate()
print("Result folder creation error (if any):", folderCreated, error)

Result folder creation error (if any): b'Asset projects/ee-open-natural-ecosystems/assets/one10m/finish2025_satEmbedTs/yearwiseClassify_AgroecolZonesNum_finalPred_fullAoi already exists.\n' None


Create an ImageCollection within the results folder to hold the classifier predictions.

In [6]:
createResCollCmd = f"earthengine create collection {homeFolder}{resFolderAgroecolZonesNum_finalPred}/{resColl}"
process = subprocess.Popen(createResCollCmd.split(), stdout=subprocess.PIPE)
collCreated, errorColl = process.communicate()
print("Result image collection creation error (if any):", collCreated, errorColl)

Result image collection creation error (if any): b'Asset projects/ee-open-natural-ecosystems/assets/one10m/finish2025_satEmbedTs/yearwiseClassify_AgroecolZonesNum_finalPred_fullAoi/satEmbed_preds_fullAoiExp_fullModel_tiled already exists.\n' None


#### Hierarchical classification - using explicit hierarchy

Run the hierarchical classification year-wise.

In [7]:
hierarchOpts_exp = dict(coarserLevelLabelColumn = "labelL1", finerLevelLabelColumn = "labelL2")
globalModelGbtBiomesZonesOheRes_heir_master_explicit = classifyAndAssess.trainAndPredictHierarchical_master( \
    hierarchOpts_exp, \
    fOpts_globalModelGbt_agroecolZones, \
    cOpts_gbt150, \
    resultNewFolderName = resFolderAgroecolZonesNum_finalPred, \
    resultNewCollName = resColl, \
    returnExisting = False)
print(globalModelGbtBiomesZonesOheRes_heir_master_explicit.getInfo())
# print(globalModelGbtBiomesZonesOheRes_heir_master_explicit.size())

====== 2017 ======
projects/ee-open-natural-ecosystems/assets/one10m/finish2025_satEmbedTs/sampledFeatures_balanced/pointsWithFeaturesYearwise_2017 points path
l0 names & codes: {'nonone': 100, 'one': 200}
projects/ee-open-natural-ecosystems/assets/one10m/finish2025_satEmbedTs/sampledFeatures_balanced/pointsWithFeaturesYearwise_2017  year points asset folder
['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63', 'ae_regcode'] selected features
exporting ... 2017 l0
Total tiles 24
Started Task:  1
Started Task:  2
Started Task:  3
Started Task:  4
Started Task:  5
Started Task:  6
Started Task:  7
St

##### Combine hierarchical probabilities

Combine each year's hierarchical probabilities using explicit probability hierarchy, to get year-wise probabilistic classification for all classes.

In [7]:
combinedOutputs = classifyAndAssess.annualHierarchicalPredictions(
    resultNewFolderName = resFolderAgroecolZonesNum_finalPred, resultNewCollName = resColl)
print(combinedOutputs.getInfo())

projects/ee-open-natural-ecosystems/assets/one10m/finish2025_satEmbedTs/yearwiseClassify_AgroecolZonesNum_finalPred_fullAoi/satEmbed_preds_fullAoiExp_fullModel_tiled predHeirProbsColl id
exporting expHierMult for year 2017...
Total tiles 24
Started Task:  1
Started Task:  2
Started Task:  3
Started Task:  4
Started Task:  5
Started Task:  6
Started Task:  7
Started Task:  8
Started Task:  9
Started Task:  10
Started Task:  11
Started Task:  12
Started Task:  13
Started Task:  14
Started Task:  15
Started Task:  16
Started Task:  17
Started Task:  18
Started Task:  19
Started Task:  20
Started Task:  21
Started Task:  22
Started Task:  23
Started Task:  24
exporting expHierMult for year 2018...
Total tiles 24
Started Task:  1
Started Task:  2
Started Task:  3
Started Task:  4
Started Task:  5
Started Task:  6
Started Task:  7
Started Task:  8
Started Task:  9
Started Task:  10
Started Task:  11
Started Task:  12
Started Task:  13
Started Task:  14
Started Task:  15
Started Task:  16
Sta

#### Find max probability timeseries label sequence

Using the year-wise probabilistic classifications above along with region-wise class transition probability matrices in `aer_hmm_parameters.ini`, find the optimal label sequence (Viterbi decoding) for 7 years (2018 - 2024) by statistically mitigating spurious class transitions between years.

This step frames the annual landcover classification problem as a hidden Markov model (HMM) and uses Viterbi decoding to arrive at the best label sequence.

In [7]:
classifyAndAssess.optimalTimeseriesLabels(resultFolderName = resFolderAgroecolZonesNum_finalPred)

------ Step 1 ------
------ Steps 2-7 ------
7 stepsDone
projects/ee-open-natural-ecosystems/assets/one10m/finish2025_satEmbedTs/yearwiseClassify_AgroecolZonesNum_finalPred_fullAoi/satEmbed_preds_fullAoiExp_fullModel_tiled/optLabelSeqInBands result export coll
Total tiles 24
Started Task:  1
Started Task:  2
Started Task:  3
Started Task:  4
Started Task:  5
Started Task:  6
Started Task:  7
Started Task:  8
Started Task:  9
Started Task:  10
Started Task:  11
Started Task:  12
Started Task:  13
Started Task:  14
Started Task:  15
Started Task:  16
Started Task:  17
Started Task:  18
Started Task:  19
Started Task:  20
Started Task:  21
Started Task:  22
Started Task:  23
Started Task:  24
